Dieses Notebook zeigt, wie man ein spezielles Rekurrentes Neuronales Netz (LSTM) zur Vorhersage von Energieverbrauchsdaten trainieren kann. 
Hierfür werden die Bibliotheken keras, sowie sci-kit learn verwendet.
Das Notebook orientiert sich an folgendem Tutorial: https://www.elab2go.de/demo-py5/ (Autoren: Prof. Dr. Eva Maria Kiss, B. Sc. Franc Willy Pouhela, M. Sc. Anke Welz)

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import math

# matplotlib und seaborn zum Plotten
import matplotlib.pyplot as plt
import seaborn as sns

# scikit-learn für Überwachtes Lernen
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import max_error

# keras für Neuronale Netze
import tensorflow as tf 
import keras as keras
from keras.models import Sequential 
from keras.layers import Dense, LSTM, Dropout 
from keras.utils.vis_utils import plot_model 
from keras import activations

# Für das Darstellen von Bildern im SVG-Format
import graphviz as gv
import pydot
from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG

In [ ]:
# Daten einlesen
data = pd.read_csv('https://www.elab2go.de/demo-py5/opsd_2016-2019.csv')

In [ ]:
data.head()

In [ ]:
data.tail()

# Aufgabe 1: Datenvorbereitung
 

* a) Überprüfen Sie, ob in dem Datensatz NaN enthalten sind. Falls ja, überlegen Sie sich, wie Sie damit am besten umgehen.
*   b) Ist es sinnvoll, mit allen Merkmalen fortzufahren? Was müssen Sie tun um diese Frage beantworten zu können?
*   c) Bringen Sie die Spalte "Datum" in ein geeignetes Format und indizieren Sie das dataframe mit dieser Spalte 




In [ ]:
# Frage 1 a) Überprüfen auf NaN

# Um NaN-Werte im Datensatz zu überprüfen, können Sie die Methode data.isnull().sum() verwenden.
# Wenn es NaN-Werte gibt, können Sie sie entweder durch geeignete Werte ersetzen (z. B. den Mittelwert der Spalte)
# oder die Zeilen, die NaN-Werte enthalten, entfernen.

In [ ]:
# Frage 1 b)

#Um zu entscheiden, ob es sinnvoll ist, mit allen Merkmalen fortzufahren, 
#sollten Sie eine explorative Datenanalyse durchführen 
#und die Korrelationen zwischen den Merkmalen und dem Ziel (Energieverbrauch) untersuchen. 
#Sie könnten auch Modelle mit unterschiedlichen Merkmalssätzen trainieren und ihre Leistung vergleichen, 
#um zu entscheiden, welche Merkmale am besten zur Vorhersage des Energieverbrauchs beitragen.



In [ ]:
# ...

In [ ]:
# Frage 1 c)

# Um das Datumsformat zu ändern und den DataFrame damit zu indizieren, verwenden Sie:
data['Datum'] = pd.to_datetime(data['Datum'])
data.set_index('Datum', inplace=True)

In [ ]:
# Plotten des Stromverbrauchs. Der Stromverbrauch soll später mithilfe des Neuronalen Netzes vorhergesagt werden.

sns.set(rc={'figure.figsize':(12, 4)})
sns.set_color_codes('bright')

ax = data['Verbrauch'].plot(linewidth=1, color='b', marker = '.')
ax.set_title('Täglicher Stromverbrauch')
ax.set_xlabel('Datum');
ax.set_ylabel('GwH');

# Aufgabe 2

Im nächsten Codeblock werden zeitlich versetzte Merkmale (sogenannte Lagged Features) erzeugt.  

*   a) Was macht dabei die Funktion shift()? Schauen Sie in der pandas Dokumentation nach!
*   b) Was macht die Funktion concat()? Schauen Sie auch das in der pandas Dokumentation nach!
*   c) Führen Sie die untenstehende Zelle aus und Lassen Sie sich anschließend die ersten 10 Zeilen des neu erzeugten dataframes ausgeben. Hat das Erzeugen der zeitlich versetzten Merkmale erfolgreich funktioniert?



# Antworten



* a) Die Funktion shift() verschiebt die Werte in einer pandas Series oder einem DataFrame entlang der angegebenen Achse um eine     bestimmte Anzahl von Perioden. In diesem Fall wird sie verwendet, um zeitlich versetzte Merkmale (Lagged Features) zu      erzeugen.
*   b) Die Funktion concat() verbindet pandas-Objekte (Series oder DataFrames) entlang einer bestimmten Achse.
*   c) Um die ersten 10 Zeilen des neu erzeugten DataFrames anzuzeigen, verwenden Sie data.head(10).


In [ ]:
# Erzeuge zeitlich versetzte Merkmale (lagged features)

consumption = pd.DataFrame(data['Verbrauch'])

data = pd.concat([consumption.shift(7),consumption.shift(6), consumption.shift(5), consumption.shift(4), 
                  consumption.shift(3), consumption.shift(2), consumption.shift(1), data[['Verbrauch']]], axis=1)

data.columns =  ['t-7', 't-6', 't-5', 't-4', 't-3', 't-2', 't-1', 'Verbrauch']

data.head()

data.dropna(inplace=True)

In [ ]:
data.head(10)

In [ ]:
# Daten in Trainings- und Testset aufspalten

TEST_SPLIT = 0.1 
#data = data.drop(columns = ['Wind', 'Solar'])
train_size = int(len(data) * (1-test_split))
test_size = len(data) - train_size
train = data.iloc[0:train_size]
test = data.iloc[train_size:len(data)]
print("Trainingsdaten:") 
print(train.head())
print(train.tail())
print("Testdaten:") 
print(test.head())
print(test.tail())

In [ ]:
# Daten auf den Wertebereich [0, 1] skalieren

scaler = MinMaxScaler(feature_range = (0,1))

train_s = scaler.fit_transform(np.array(train))
test_s = scaler.fit_transform(np.array(test))

print("Trainingsdaten (unskaliert)\n")
print(train.head(3))
print("\nTrainingsdaten (skaliert)\n")
train_s = pd.DataFrame( train_s , columns = data.columns)
train_s = train_s.set_index(train.index )
print(train_s.head(3))

print("Testdaten (unskaliert)\n")
print(test.head(3))
print("\nTestdaten (skaliert)\n")
test_s = pd.DataFrame( test_s , columns = data.columns)
test_s = test_s.set_index(test.index )
print(test_s.head(3))

In [ ]:
# In Merkmale und Label aufteilen

X_train = train_s.drop(columns = ['Verbrauch'])
y_train = pd.DataFrame(train_s['Verbrauch'])

print(X_train.head())
print(y_train.head())

In [ ]:
TIMESTEPS = 7 # Länge eines gleitenden Zeitfensters
UNITS = 10 # Ausgabedimension einer einzelnen LSTM-Schicht
N_LAYER = 2 # Anzahl an LSTM-Schichten

# Initialisiere ein sequentielles Modell (Modell mit mehreren Schichten)
model = Sequential(name='sequential') 

#Füge so viele Schichten hinzu, wie in N_LAYER angegeben
for i in range(N_LAYER):

  lstm_layer = LSTM(units = UNITS, # Dimension der Ausgabe
                    input_shape=(TIMESTEPS,1), # Dimension der Eingabe (1. Dimension entspricht Anzahl Zeitschritte, 2. Dimension:  1, da nur ein Merkmal = Verbrauch über die Zeit betrachtet wird (univariates Problem) )
                    return_sequences=True, # wir wollen die gesamte Ausgabesequenz der jeweiligen LSTM-Schicht in die nächste LSTM-Schicht weitergeben können
                    name = 'lstm_' + str(i+1)) # Schichten werden mit "lstm_" + Schichtnummer benannt
                   
  model.add(lstm_layer) # Schicht dem Modell hinzufügen


# weitere LSTM Schicht hinzufügen, bei der nur die letzte Ausgabe (und nicht wie oben die ganze Sequenz) in die nächste Schicht weitergegeben wird
model.add(LSTM(units = UNITS, input_shape=(TIMESTEPS,1), name = 'lstm_' + str(N_LAYER+1)))

# Lineares Layer mit ReLU-Aktivierungsfunktion
model.add(Dense(units = 1, name='dense_1'))#, activation=activations.tanh))

# Konfiguriere das Modell für die Trainingsphase 

# Angeben welcher Optimierer verwendet werden soll
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

# über welche Verlustfunktion optimiert werden soll 
# und wie die Performance des Modells gemessen werden soll (hier werden Verlustfunktion und Performancemetrik gleich gewählt, muss aber nicht so sein))
model.compile(optimizer = opt, loss = "mse", metrics=['mean_squared_error'])

# Zusammenfassung und Visualisierung des Modells
model.summary()
plot_model(model, show_shapes=True, show_layer_names=True)

# Aufgabe 3

Welchen Parameter der obigen Codezelle müssten Sie verändern, um 3 LSTM Schichten zu bekommen?

->Um 3 LSTM-Schichten im Modell zu erhalten, ändern Sie den Wert von N_LAYER auf 3.

# Aufgabe 4

Das Neuronale Netz wird in der unten stehenden Codezelle trainiert. Führen Sie den Code aus. 
Nach wie vielen Epochen wird das Training auf jeden Fall stoppen?

-> Das Training wird auf jeden Fall nach der Anzahl von Epochen stoppen, die in der Variable EPOCHS angegeben ist. In diesem Fall beträgt die Anzahl der Epochen 1000.

In [ ]:
# Modell trainieren

# Erstelle Callback für Stop-Kriterium
from keras.callbacks import EarlyStopping, CSVLogger
cb_stop = EarlyStopping(monitor='val_loss', mode='min', 
                        verbose=1, patience=200)
log_file = 'demo-py5-log.csv'
cb_logger = CSVLogger(log_file, append=False, separator=';')

# X_train erhält eine zusätzliche Dimension und wird dreidimensional
X_train = np.array(X_train)
X_train = np.reshape(X_train, 
                     (X_train.shape[0], X_train.shape[1], 1))

print(X_train.shape)

# Trainiere das Modell mit Hilfe der Funktion fit()
BATCH_SIZE = 64
EPOCHS = 1000
history = model.fit(X_train, y_train, 
                    epochs=EPOCHS, batch_size=BATCH_SIZE, 
                    validation_split=TEST_SPLIT, verbose=2, 
                    callbacks=[cb_logger, cb_stop])

# Speichere das Modell im Format HDF5
model.save("model_adam.h5")

print("History");print(history.history.keys());

# Aufgabe 5

Im untenstehenden Codeblock wird der Trainingsfortschritt geplottet. Wie beurteilen Sie ihn?

In [ ]:
# Trainingsfortschritt plotten

plt.plot(history.history['mean_squared_error'], label='MSE (Trainingsdaten)')
plt.plot(history.history['val_mean_squared_error'], label='MSE (Testdaten)')

plt.title('Training: Entwicklung des Fehlers')
plt.ylabel('MSE-Fehler')
plt.xlabel('Epochen')
plt.legend()

In [ ]:
# Testdaten in Merkmale X und Label y aufteilen

X_test = test_s.drop(columns = ['Verbrauch'])
y_test = pd.DataFrame(test_s['Verbrauch'])

# Vorhersage für skalierte Testdaten
X_test = np.array(X_test)
X_test_input = np.reshape(X_test, 
        (X_test.shape[0], X_test.shape[1], 1))

y_test = np.array(y_test)
#y_test = np.reshape(y_test, 
        # (y_test.shape[0],  1))
y_pred = model.predict(X_test_input)

pred = np.concatenate((X_test, y_pred), axis=1)

# Reskaliere die Daten
test_rs = pd.DataFrame(scaler.inverse_transform(test_s), columns = test_s.columns)
pred_rs = pd.DataFrame(scaler.inverse_transform(pred), columns = test_s.columns)

y_test = test_rs['Verbrauch']
y_pred = pred_rs['Verbrauch']

# Berechne RMSE der Validierungsdaten
mse = mean_squared_error(y_test, y_pred)
rmse = np.round(np.sqrt(mse))
print("Validierungs-Fehler:")
print("\nMSE:\n %.2lf" % (mse))

In [ ]:
def rel_error(df, col1, col2):
    df['Error (%)'] = (df[col1] - df[col2]) / df[col1]  * 100
    df['Error (%)'] = df['Error (%)'].abs()
    return df['Error (%)']

In [ ]:
# Hilfsfunktion error_table erzeugt Fehlertabelle
def error_table(df1, df2, col1, col2, idx):
    cols = [df1, df2]
    headers = [col1, col2]
    # Erzeuge pred aus y_test und y_pred mit Index idx
    pred = pd.concat(cols, axis=1, keys=headers) 
    pred.set_index(idx, inplace=True)
    # Füge Fehler-Spalten hinzu
    pred['Error'] = np.abs(pred['y_test'] - pred['y_pred'])
    pred['Error(%)'] = rel_error(pred, 'y_test', 'y_pred')
    pred = pred.astype(float).round(1)
    # Füge RMSE hinzu
    mse = mean_squared_error(df1, df2)
    rmse = np.round(np.sqrt(mse))
    pred.index.name = "RMSE: " + str(rmse)
    return pred

# Erzeuge Fehlertabelle für Validierung und gebe sie aus
idx = test.index;
pred = error_table(pd.DataFrame(y_test), pd.DataFrame(y_pred), 'y_test', 'y_pred', idx)

pred.head()

In [ ]:
pred['Error (%)'].min()

In [ ]:
pred['Error (%)'].max()